# T1 SEMANTIC SEGMENTATION

Import and prepare point clouds for semantic segmentation and do the inference.
To run these scripts, create a python 3.10 environment & install geomapi (numpy, opend3d, ifcopenshell, trimesh, ...), pytorch

## LIBRARIES

In [57]:
#IMPORT PACKAGES
from pathlib import Path
import os
import sys
import numpy as np
import laspy
from geomapi.utils import geometryutils as gmu
import torch
import os
from collections import OrderedDict
import time
import torch.nn.functional as F
import json
#CONTEXT
import context 

#UTILS
from utils.t1_utils import *

In [58]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
%autoreload 2

## INPUT

In [60]:
path= Path(os.getcwd()).parents[0]/'data'/'t1_data_test'
print( path)
point_cloud_path =path/'input'/'Eva.las'
input_folder =  path/'input'

#TRAINING


#INFERENCE
config_path = path/'config.py' 
weights = path/'model'/'model_best.pth' 


#RESULTS
class_file=path.parents[0]/'_classes.json'
labels_path = path/'result'/'Eva_pred.npy' # -> pred_save_path
save_path =  path/'result'
os.makedirs(save_path) if not save_path.exists() else None

/home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1_data_test


In [61]:
#parse config file
cfg = default_config_parser(str(config_path), 
                            {'save_path': str(path), 
                             'weight': str(weights)})

In [62]:
# #or directly run python from main repo
# export PYTHONPATH="/home/mbassier/code/Scan-to-BIM-CVPR-2024/thirdparty:$PYTHONPATH" # run this if submodule is not found
# python thirdparty/pointcept/pointcept/tools/inference_kul.py --config-file /home/mbassier/code/Scan-to-BIM-CVPR-2024/thirdparty/pointcept/configs/kul/kul-pt-v3-base.py --options save_path=/home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1_data_test/result weight=/home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1_data_test/model/model_best.pth


## INPUT DATA CONVERSION

Preprocessing of input data

In [63]:
scene_id = os.path.basename(point_cloud_path)
name, ext = os.path.splitext(scene_id)

#Read LAS/LAZ
las = laspy.read(point_cloud_path)
print(list(las.point_format.dimension_names))

pcd = gmu.las_to_pcd(las)
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()

#populate dict
coords = np.stack([las.x, las.y, las.z], axis=1)
colors=np.asarray(pcd.colors)
normals = np.asarray(pcd.normals)    
save_dict = dict(coord=coords, color=colors, normal=normals, scene_id=scene_id) #, semantic_gt=las.labels.astype(int))

#save in input folder
torch.save(save_dict, os.path.join(input_folder, f"{name}.pth"))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue']


## INFERENCE

Inference using Point Transformer V3

In [64]:
#POINTCEPT
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.engines.test import TESTERS
from pointcept.engines.launch import launch
from pointcept.datasets import build_dataset, collate_fn
import pointcept.utils.comm as comm

from pointcept.engines.defaults import create_ddp_model

from pointcept.utils.misc import make_dirs
from pointcept.models import build_model

def collate_fn(batch):
    return batch

def build_inference_model(cfg):
    model = build_model(cfg.model)
    n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    model = create_ddp_model(
        model.cuda(),
        broadcast_buffers=False,
        find_unused_parameters=cfg.find_unused_parameters,
    )
    if os.path.isfile(cfg.weight):
        checkpoint = torch.load(cfg.weight)
        weight = OrderedDict()
        for key, value in checkpoint["state_dict"].items():
            if key.startswith("module."):
                if comm.get_world_size() == 1:
                    key = key[7:]  # module.xxx.xxx -> xxx.xxx
            else:
                if comm.get_world_size() > 1:
                    key = "module." + key  # xxx.xxx -> module.xxx.xxx
            weight[key] = value
        model.load_state_dict(weight, strict=True)

    else:
        raise RuntimeError("=> No checkpoint found at '{}'".format(cfg.weight))
    return model

def main_worker(cfg):    
    cfg = default_setup(cfg)
    test_dataset = build_dataset(cfg.data.test)
    
    if comm.get_world_size() > 1:
        test_sampler = torch.utils.data.distributed.DistributedSampler(test_dataset)
    else:
        test_sampler = None
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=cfg.batch_size_test_per_gpu,
        shuffle=False,
        num_workers=cfg.batch_size_test_per_gpu,
        pin_memory=True,
        sampler=test_sampler,
        collate_fn=collate_fn,
    )
    
    
    model = build_inference_model(cfg)
    model.eval()
   
    for idx, data_dict in enumerate(test_loader):
        data_dict = data_dict[0]  # current assume batch size is 1
        fragment_list = data_dict.pop("fragment_list")
        segment = data_dict.pop("segment")
        data_name = data_dict.pop("name")
        pred_save_path = os.path.join(save_path, "{}_pred.npy".format(data_name))

        pred = torch.zeros((segment.size, cfg.data.num_classes)).cuda()
        for i in range(len(fragment_list)):
            fragment_batch_size = 1
            s_i, e_i = i * fragment_batch_size, min(
                (i + 1) * fragment_batch_size, len(fragment_list)
            )
            input_dict = collate_fn(fragment_list[s_i:e_i])[0]
            for key in input_dict.keys():
                if isinstance(input_dict[key], torch.Tensor):
                    input_dict[key] = input_dict[key].cuda(non_blocking=True)
            idx_part = input_dict["index"]            
            with torch.no_grad():
                pred_part = model(input_dict)["seg_logits"]  # (n, k)
                pred_part = F.softmax(pred_part, -1)
                if cfg.empty_cache:
                    torch.cuda.empty_cache()
                bs = 0                
                for be in input_dict["offset"]:
                    pred[idx_part[bs:be], :] += pred_part[bs:be]
                    bs = be        
        pred = pred.max(1)[1].data.cpu().numpy()
        np.save(pred_save_path, pred)

    print("DONE.")


launch(
    main_worker,
    num_gpus_per_machine=1,
    num_machines=1,
    machine_rank=0,
    dist_url='auto',
    cfg=(cfg,),
)

[2024-04-09 13:52:01,715 INFO kul.py line 58 735719] Totally 1 x 1 samples in input set.


test_sampler is None
DONE.


## RESULTS

import classes

In [65]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

# Creating a dictionary to map temp_id to id
temp_id_to_id = {cls["temp_id"]: cls["id"] for cls in class_dict["classes"]}

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Convert inference result back to .las

In [67]:
import numpy as np

# las = laspy.read(point_cloud_path)

# Remapping function using numpy's vectorize to map temp_ids to ids
id_mapper = np.vectorize(temp_id_to_id.get)
temp_ids =np.load(labels_path)
labels = id_mapper(temp_ids)


las.add_extra_dim(laspy.ExtraBytesParams(
    name="labels",
    type=np.int32
))

las["labels"] = labels

las.write(os.path.join(save_path, "Eva_pred.las"))